In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

path = 'C:/Users/queenSSH/Desktop/학교/CUAI/cuai하계/'

In [2]:
naver = pd.read_csv(path + '네이버_추천시스템.csv' , encoding='cp949')

In [3]:
daum = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')

In [4]:
naver.head()

,Unnamed: 0,title,author,genre,platform,story,rate
0,0,마물전담부대,염손,"스토리, 판타지",네이버 웹툰,마법혁명으로 근대화를 이룬 판타지 세계.\n하지만 이러한 세계에서도 마물은 여전히 ...,9.11
1,1,불릿,김도근,"스토리, 스포츠",네이버 웹툰,"축구부 에이스, 좋아하는 선배에게 고백하려 했는데... 에어소프트 게임 마스터가 된다?!",9.89
2,2,더,공현곤,"스토리, 스릴러",네이버 웹툰,수천년 전 조상들의 경주로 정해진 12인의 운명. 자리를 지키려는 1등과 바꾸려는 ...,9.94
3,3,각자의,오묘,"스토리, 로맨스",네이버 웹툰,헤어졌지만 헤어지지 않은 사이\n솔직하지 못한 우리들 어떻게 될까요?,9.97
4,4,노선도,효기,"스토리, 스릴러",네이버 웹툰,암호화폐 투자 실패로 모든 걸 잃은 평범한 40대 회사원 '권한수'.\n절망한 그의...,9.82


In [11]:
daum.tail()

,title,genre,story
802,항해1,"액션, 판타지, 초능력",작가 최종훈(hun)의 세 번째 웹만화 '항해'!
803,해골택시,"판타지, 코믹, 죽음","이제 스스로 삶을 포기하려는 자여, 당신 앞으로 택시 한 대가 멈춰 설 것이다."
804,천사의 섬,"코믹, 판타지, 서바이벌","하염없이 섬에서 표류하는 주인공. 그러던 어느날, 그의 앞에 천사가 나타난다."
805,울트라병장,"판타지, 코믹, 영웅",리얼코믹 SF메카닉 만화. 울트라 병장은 대한민국의 평화를 지킬수 있을 것인가
806,스멜스 라이크 30 스피릿,"일상, 코믹, 음악",고리타 작가 특유의 개그코드로 우울한 현대인의 자화상을 즐겁게 '편곡'한 작품


In [6]:
naver = naver[['title', 'genre','story']]
daum = daum[['title', 'genre','story']]

In [8]:
web_combined = pd.concat([naver, daum], axis=0, ignore_index=True)

In [10]:
web_combined.tail()

,title,genre,story
2225,항해1,"액션, 판타지, 초능력",작가 최종훈(hun)의 세 번째 웹만화 '항해'!
2226,해골택시,"판타지, 코믹, 죽음","이제 스스로 삶을 포기하려는 자여, 당신 앞으로 택시 한 대가 멈춰 설 것이다."
2227,천사의 섬,"코믹, 판타지, 서바이벌","하염없이 섬에서 표류하는 주인공. 그러던 어느날, 그의 앞에 천사가 나타난다."
2228,울트라병장,"판타지, 코믹, 영웅",리얼코믹 SF메카닉 만화. 울트라 병장은 대한민국의 평화를 지킬수 있을 것인가
2229,스멜스 라이크 30 스피릿,"일상, 코믹, 음악",고리타 작가 특유의 개그코드로 우울한 현대인의 자화상을 즐겁게 '편곡'한 작품


In [14]:
tfidf = TfidfVectorizer()

#story에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(web_combined['story'])
print(tfidf_matrix.shape)

(2230, 16221)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix.shape

(2230, 2230)

In [17]:
np.round(cosine_matrix,4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [20]:
#web title와 id를 매핑할 dictionary 생성

web2id = {}
for i, c in enumerate(web_combined['title']): 
    web2id[i] = c


#id와 web 매핑할 dictionary 생성
id2web = {}
for i,c in web2id.items() : 
    id2web[c] = i

In [21]:
title_list = web_combined['title']

In [22]:
#마물전담부대의 id 추출

for j in range(len(title_list)):
    #idx = id2web['마물전담부대'] #마물전담부대 :  0번 인덱스
    idx = id2web[title_list[j]]
    print(web2id[j])
    sim_scores = [(i,c) for i,c in enumerate(cosine_matrix[idx]) if i != idx]
    #자기 자신을 제외한 영화들의 유사도 및 인덱스 추출

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse = True) #유사도가 높은 순서대로 정렬
    sim_scores[0:10] #상위 10개의 인덱스와 유사도 추출
    
    sim_scores = [(web2id[i], score) for i, score in sim_scores[0:10]]
    print(sim_scores)

마물전담부대
[('아수라발발타', 0.10058018941110215), ('화이트멜로우', 0.09587701251182498), ('SM', 0.0768966844657681), ('송곳', 0.07517086866566873), ('참교육', 0.07478232458601769), ('번개기동대', 0.07058719867254608), ('투엔티스', 0.0705483486461409), ('욕망일기', 0.06667881358258841), ('남과', 0.06572899567390852), ('크레이지', 0.06548907762854013)]
불릿
[('여자만화 구두', 0.14805099022568233), ('스마트폰', 0.1220414232034521), ('얼굴천재', 0.08540461360907212), ('우리집에', 0.08364737049869526), ('리와 구', 0.0818801947690296), ('에비츄', 0.08044040715275168), ('하드캐리', 0.07927271660594504), ('같은도장', 0.07634685048634778), ('돼지만화컷툰휴재', 0.07433293449621856), ('방', 0.0720662019222241)]
더
[('대작', 0.1235139165718419), ('꼬리', 0.1171897748356099), ('서북의', 0.09625096210928519), ('경성야상곡', 0.09381263401309933), ('이스크라', 0.08881002592811632), ('신선비', 0.08693496556545852), ('창백한 말', 0.08452979979750598), ('소쩍이 운다', 0.0688966134268436), ('사우러스', 0.06603696946454478), ('너의 미소가 함정', 0.06100695942239721)]
각자의
[('잡다한컷컷툰', 0.2516388569461043), ('이별만화', 0.11777911475

[('천국의', 0.25953132198970263), ('페이머스맨', 0.1513331838409463), ('신상', 0.14301973814353836), ('이끼', 0.12994217241181646), ('피치 (PITCH)', 0.11815383029055312), ('드래곤레시피', 0.10362199523934955), ('미래의', 0.10015211299530648), ('커피우유신화', 0.09089724780409433), ('라서드', 0.09068717263582521), ('주말 도미 시식회', 0.08841832116539343)]
하루
[('커피와하루', 0.1873436845991318), ('새와', 0.1706494679183735), ('10월', 0.14693686829401945), ('당신의 모든 순간', 0.13826313618706543), ('소곤소곤', 0.1345587003004155), ('동네변호사', 0.12645632923703384), ('신사의 집', 0.1264375027710009), ('요괴대전', 0.12557497774253717), ('연애학', 0.11535942241946982), ('국민사형투표', 0.10445583600312362)]
김왕짱
[('뿔', 0.0794809582541302), ('미완결', 0.06515956801954494), ('우리동네에 왜 왔니', 0.06216138608628821), ('마녀', 0.06179828755517076), ('아이들의', 0.060382289159079665), ('사상최강', 0.058123632560038295), ('어쩔꼰대', 0.05731986139850031), ('피그말리온 컴플렉스', 0.056129757349917625), ('루드비코의 만화영화', 0.054600605488672856), ('적당한 온도', 0.05421630274249398)]
킬링타임
[('망자의 서', 0.193219801223100

[('커피와하루', 0.1873436845991318), ('새와', 0.1706494679183735), ('10월', 0.14693686829401945), ('당신의 모든 순간', 0.13826313618706543), ('소곤소곤', 0.1345587003004155), ('동네변호사', 0.12645632923703384), ('신사의 집', 0.1264375027710009), ('요괴대전', 0.12557497774253717), ('연애학', 0.11535942241946982), ('국민사형투표', 0.10445583600312362)]
환생동물학교
[('크레이터', 0.1500761421107904), ('크리스마스는 쨈과 함께', 0.10673461116688537), ('하우스키퍼', 0.0856950205614781), ('관계의', 0.0749526400612924), ('얌얌숲 당근길 토요요', 0.07048350816489708), ('별신마을 각시', 0.06546615461031702), ('장마', 0.06399594680678697), ('사랑의 헌옷수거함', 0.06342693226559917), ('위 아 애니멀스', 0.06170052556785101), ('선택은 하나', 0.06056900625519152)]
감염자
[('괴물 같은 아이돌', 0.23271693097094812), ('무늬만 사람', 0.1329150406997053), ('괴물아기', 0.12586087476323182), ('피와 나비', 0.12432791468923765), ('플레이어', 0.11863968852309323), ('피와', 0.11231063927587914), ('노동본색', 0.1054879756956559), ('블랙수트', 0.10507323511325445), ('재벌과의 인터뷰', 0.10456611550650016), ('호구의 사랑', 0.09904669296960422)]
지금
[('개같은 세상', 0.109

[('퍼펙트 마인', 0.14113215977433852), ('다시, 봄', 0.10959217142170238), ('SM', 0.09717399446052938), ('커피와 하루 the start', 0.0966783423878129), ('마왕의 성', 0.09644666738676291), ('송곳', 0.09499308879256906), ('나는', 0.09289229476082211), ('점핑걸', 0.09221033610962447), ('R에 관해서', 0.08997048977331187), ('번개기동대', 0.08920072562873357)]
빙의
[('뿔', 0.4709612262067198), ('악령주의보', 0.3080276939330258), ('데모니악', 0.28199580572875294), ('죽어도', 0.24633222040495373), ('먹이', 0.24302001630655362), ('오!주예수여', 0.21021787792946411), ('리즌', 0.1951565760089052), ('저승사자 출입금지', 0.1782147255107584), ('블라인드', 0.15796578213789003), ('한줌물망초', 0.13608619805780806)]
미선
[('심부름센터', 0.1912710638236451), ('우리학교 김선생님', 0.18613055401016035), ('체벌교사', 0.10804264768783378), ('알파', 0.09545334722901078), ('앵무살수', 0.09231898457624471), ('허약선생', 0.09201060487262708), ('특별직', 0.09153737733645757), ('목욕의', 0.08955836455690817), ('곱게 자란 자식', 0.08798151748263033), ('죽는남자', 0.08751989931758943)]
뱀파이어
[('노숙자 블루스', 0.11647362390882314), ('PEAK',

탐구생활2
[('탐구생활4-그대와', 0.14470310906041903), ('몽연', 0.1276278447204939), ('용의주도 황비', 0.10559767126082295), ('화양연화', 0.09966894592085908), ('과격자매단', 0.09911063438842507), ('트라이앵글', 0.09164403357087221), ('버디스', 0.07637071607460572), ('룬의 이야기', 0.07555284576640203), ('반짝반짝 컬링부', 0.07345257357391771), ('침묵의 정원', 0.07294644857475086)]
구석구석캠페인
[('네이버', 0.13981817059676468), ('화양연화', 0.12480064606655676), ('2014', 0.11437745652184336), ('마음이 만든 것', 0.10596868697958259), ('MY', 0.09972837309942666), ('2011', 0.09568085071400298), ('개편', 0.08849794032339421), ('미숙한', 0.08605500533714058), ('2012', 0.08547891419440427), ('2015', 0.08453881191351843)]
체스아일
[('흐드러지다', 0.1590086892249083), ('이웃잔혹사', 0.1374165987998761), ('고향의꽃', 0.13654106892349677), ('하이드', 0.12396012791613586), ('주말 도미 시식회', 0.10139458535848593), ('헬58', 0.10059291306648013), ('들숨날숨', 0.09343829390805189), ('닥터', 0.08919184435827815), ('봄이여', 0.0882207835887725), ('나는', 0.08090297494809927)]
펫다이어리
[('커피와 하루 the start', 0.177161433

[('죄의', 0.131024056338421), ('개편', 0.07085916102374468), ('단군할배요', 0.07038662749598502), ('연옥님이', 0.06123685105739648), ('폐어', 0.05974019063161248), ('지옥사원', 0.059493636385740706), ('사냥개들', 0.05918125776383997), ('장마', 0.05691412926655652), ('별의', 0.05530338562568139), ('도깨비언덕에 왜 왔니?', 0.05513492800282141)]
전설의 화석
[('전설의', 0.06226940374658574), ('치원 신드롬', 0.05682258203456106), ('주욱 같은 하루', 0.05280062172649496), ('소년들은', 0.04928351677056103), ('에어리어제로', 0.04924977046199072), ('삶이', 0.04816629267627141), ('조숙의 맛', 0.04619069531322755), ('집떠난 개고생', 0.04587128236799392), ('열무와 알타리', 0.04533385329654853), ('바른연애 길잡이', 0.04528817498465039)]
구름이 피워낸 꽃
[('여혜', 0.13540829756109246), ('가슴도 리콜이 되나요', 0.10757344229426112), ('그림자정원', 0.08975824986490794), ('작약만가 : 서리꽃', 0.08965843991305035), ('하루만 네가 되고 싶어', 0.08916612072271145), ('별의', 0.0878365338031387), ('룬의 이야기', 0.0865750578282874), ('그림형제 잔혹동화', 0.08468781391786902), ('N번째 연애', 0.07512633402576364), ('사이드킥', 0.07496322042648146)]
판사 이한영
[('체

[('혼자를 기르는 법', 0.12727236195493138), ('2016', 0.09089746360929009), ('로그아웃', 0.07273847906852847), ('귀전구담', 0.0696531155625846), ('레코닝', 0.06038128582754468), ('사노라면', 0.05783897373132258), ('선택은 하나', 0.052722243791959986), ('집이 없어', 0.05163262126648136), ('TEN', 0.050294314396480286), ('죽어 마땅한 자', 0.04901199597314458)]
대치동 일타강사
[('미세스 선녀', 0.14207360818857315), ('집행자여', 0.11268452986280456), ('오세요 타로샵', 0.10209029659037533), ('이기자,', 0.10136119884289829), ('요리대마왕', 0.09661248183191595), ('경운기를', 0.09403687737934241), ('위아더', 0.08336555676710702), ('감히', 0.08065144664332378), ('뱀파이어', 0.08060804441814726), ('구름의', 0.06811428818856921)]
시동
[('지옥사원', 0.18437645232211106), ('올망동은 평화롭습니다', 0.16728520089027873), ('구구까까', 0.1576244307281987), ('낢에게와요', 0.13034112413692348), ('모르모트', 0.10861763899603627), ('좀비딸', 0.10828811282501637), ('네가', 0.10564398196518883), ('노동본색', 0.1046111584392264), ('미 앤 아이', 0.1001070575680673), ('모태솔로수용소', 0.0983223691544986)]
남은 고양이
[('은밀하게 위대하게', 0.139443546665

[('신사의 집', 0.18460346485585993), ('괴물 같은 아이돌', 0.0940873021451997), ('기억흔적', 0.08609644023275438), ('크리스마스는 쨈과 함께', 0.08555034437521988), ('꽃같은 인생', 0.08247420894912981), ('떨림', 0.0812314232543844), ('102030', 0.08016359892095659), ('플레이, 플리', 0.0768152428285083), ('당신만', 0.07442251362884211), ('1인용', 0.07051311578067096)]
란의 공식
[('마물전담부대', 0.0), ('불릿', 0.0), ('더', 0.0), ('각자의', 0.0), ('노선도', 0.0), ('말하는대로', 0.0), ('연주동', 0.0), ('의도적', 0.0), ('맘마미안', 0.0), ('로맨스는', 0.0)]
코알랄라
[('복학왕', 0.26455427886415983), ('우리', 0.14208739124573688), ('멸망X초이스', 0.10822882367792125), ('찐:종합게임동아리', 0.10778350432534622), ('외발로 살다', 0.059057387355321976), ('참교육', 0.05777801791511561), ('나의', 0.05533546240886518), ('이쇼핑', 0.05479129977204965), ('라스트', 0.054694448449257575), ('그랜드', 0.05367301289671628)]
빨간봉투
[('50픽셀 데이즈', 0.16845899475361745), ('미래의', 0.15568027035975804), ('마왕을 위한 동화', 0.14710706618433023), ('프리드로우', 0.14032676479270836), ('차이니즈 봉봉클럽 북경편', 0.13953752168708175), ('멀리서 보면 푸른 봄', 0.123703558

[('뱀파이어의', 0.15125989380276375), ('화이트 블러드', 0.13349989638865437), ('두번째 집', 0.12204627075702738), ('강시대소동', 0.11271998554237478), ('찌질의', 0.106388322327978), ('루드비코의 만화일기', 0.10421230107206844), ('어쿠스틱 라이프', 0.10046590029139559), ('사라지다', 0.09719082339554509), ('복학왕', 0.09488022930137098), ('그루밍 선데이', 0.09407234221184381)]
삼풍
[('제 0시 : 대통령을 죽여라', 0.09950637829216695), ('저승사자 온라인', 0.08453197214211727), ('순정큐피트', 0.08377232234796567), ('카라멜마끼아또', 0.08264438090651183), ('왕 그리고 황제', 0.08167813015127508), ('공부하는 고양이', 0.08159326033060203), ('난약', 0.07927896471013073), ('꽃드림', 0.07461097332005498), ('소년 가라사대', 0.07408856807723654), ('괴물아기', 0.07382055711566439)]
CCTV
[('지배', 0.16640970754181456), ('웨이크 업 데드맨', 0.11324787974824463), ('모디파이휴재', 0.11167961713975107), ('상남자', 0.10859517026964273), ('아귀', 0.10584798710684035), ('죽는남자', 0.10310161151861724), ('심인애씨의 소유', 0.0992527540301128), ('나는', 0.09832906339133203), ('일진의 크기', 0.09818815257230062), ('당신의', 0.09582877716592529)]
미 앤 아이
[('난약'